<a href="https://colab.research.google.com/github/kowoonho/machine-learning/blob/main/MNIST_by_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import torch.nn.init

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(2022)

if device == 'cuda':
  torch.cuda.manual_seed_all(2022)

In [ ]:
mnist_train = dsets.MNIST(root = 'MNIST.data/', train = True, transform= transforms.ToTensor(),
                          download = True)
mnist_test = dsets.MNIST(root = 'MNIST.data/', train = False, transform= transforms.ToTensor(),
                         download = True)



In [ ]:
data_loader = torch.utils.data.DataLoader(dataset = mnist_train, batch_size = 100,
                         shuffle = True, drop_last = True)

In [ ]:
#모델 생성

class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    self.layer1 = nn.Sequential(
        nn.Conv2d(1, 32, 3, 1, 1),
        nn.ReLU(),
        nn.MaxPool2d(2,2)
    )

    self.layer2 = nn.Sequential(
        nn.Conv2d(32, 64, 3, 1, 1),
        nn.ReLU(),
        nn.MaxPool2d(2,2)
    )

    self.layer3 = nn.Sequential(
        nn.Conv2d(64, 128, 3, 1, 1),
        nn.ReLU(),
        nn.MaxPool2d(2,2)
    )

    self.fc1 = nn.Linear(128*3*3, 625, bias =True)
    torch.nn.init.xavier_uniform_(self.fc1.weight)
    self.layer4 = nn.Sequential(
        self.fc1,
        nn.ReLU(),
        nn.Dropout(0.5)
    )
    self.fc2 = nn.Linear(625, 10, bias = True)
    torch.nn.init.xavier_uniform_(self.fc2.weight)
  
  def forward(self, x):
    out = self.layer1(x)
    out = self.layer2(out)
    out = self.layer3(out)
    out = out.view(out.size(0), -1)
    out = self.layer4(out)
    out = self.fc2(out)

    return out




In [ ]:
model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 1e-3)

In [ ]:
total_batch = len(data_loader)

for epoch in range(15):
  for x, y in data_loader:
    x = x.to(device)
    y = y.to(device)


    optimizer.zero_grad()
    hypothesis = model(x)
    cost = criterion(hypothesis, y)
    cost.backward()
    optimizer.step()
    avg_cost = cost / total_batch

  
  print("Epoch: {} Cost: {:.6f}".format(
        epoch+1, cost.item()
    ))

Epoch: 1 Cost: 0.032486
Epoch: 2 Cost: 0.002023
Epoch: 3 Cost: 0.032870
Epoch: 4 Cost: 0.057067
Epoch: 5 Cost: 0.024875
Epoch: 6 Cost: 0.002489
Epoch: 7 Cost: 0.001033
Epoch: 8 Cost: 0.033605
Epoch: 9 Cost: 0.023389
Epoch: 10 Cost: 0.001504
Epoch: 11 Cost: 0.040606
Epoch: 12 Cost: 0.022656
Epoch: 13 Cost: 0.005409
Epoch: 14 Cost: 0.064259
Epoch: 15 Cost: 0.000030


In [ ]:
with torch.no_grad():
  x_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
  y_test = mnist_test.test_labels.to(device)

  prediction = model(x_test)
  correct_prediction = torch.argmax(prediction, 1) == y_test

  accuracy = correct_prediction.float().mean()

  print(accuracy.item())

/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:67: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:57: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


0.9732999801635742
